In [1]:
using ValueShapes

In [2]:
using SpecialFunctions

In [3]:
using HCubature

In [4]:
using Revise 
using BAT
using Plots
using Distributions

┌ Info: Precompiling BAT [c0cd4b16-88b7-57fa-983b-ab80aecada7e]
└ @ Base loading.jl:1260
┌ Warning: Package BAT does not have HCubature in its dependencies:
│ - If you have BAT checked out for development and have
│   added HCubature as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with BAT
└ Loading HCubature into BAT from project dependency, future warnings for BAT are suppressed.


In [30]:
NN = 5

a =  NamedTupleDist( a = GaussianShell(r=5, w=2, n=5))

samples_iid = bat_sample(a.a, 10^5).result;

┌ Info: Initializing new RNG of type Random123.Philox4x{UInt64,10}
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/algotypes/bat_default.jl:44
┌ Info: Using sampling algorithm IIDSampling()
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/algotypes/bat_default.jl:44


In [31]:
bat_integrate(samples_iid).result

┌ Info: Using integration algorithm AHMIntegration{CholeskyPartialWhitening,GeyerAutocorLen}
│   whitening: CholeskyPartialWhitening CholeskyPartialWhitening()
│   autocorlen: GeyerAutocorLen GeyerAutocorLen()
│   volumetype: Symbol HyperRectangle
│   max_startingIDs: Int64 10000
│   max_startingIDs_fraction: Float64 2.5
│   rect_increase: Float64 0.1
│   warning_minstartingids: Int64 16
│   dotrimming: Bool true
│   uncertainty: Array{Symbol}((1,))
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/algotypes/bat_default.jl:44
┌ Info: Harmonic Mean Integration started. Samples in dataset 1 / 2: 	50000 / 49999	Parameters:	5
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:93
┌ Info: Data Whitening.
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:108
┌ Info: Apply Whitening Transformation to Data Set 2
└ @ BAT /Users/vhafych/MPP-Server/gitrepos/BAT.jl/src/integration/ahmi/harmonic_me

1.0053 ± 0.0034

In [ ]:
# samples_mcmc = bat_sample(PosteriorDensity(x->LogDVal(0.0), a), (10^4, 2), MetropolisHastings()).result;

In [ ]:


true_param =(λ=zeros(NN), r=5, σ=2)

function fun(x)
	z_o = zeros(length(x))
	true_param = (λ=z_o, r=5, σ=2)
    abs_dist = sqrt(sum((z_o .- x).^2))
    return exp(-(abs_dist - true_param.r)^2/(2*true_param.σ^2)) / sqrt(2 * pi * true_param.σ^2) 
end

function LogTrueIntegral(N, true_param) 
    g(x; true_param=true_param, N=N) = x[1]^(N-1) * exp( -(x[1]-true_param.r)^2/(2*true_param.σ^2) )
    return log((sqrt(2)*pi^((N-1)/2)) / (gamma(N/2)*true_param.σ)*hcubature(g, [0], [130])[1])
end

In [ ]:
LogTrueIntegral(NN, true_param)